# #1 K_Means with Tslearn

In [ ]:
import numpy as np
from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler
from tslearn.clustering import TimeSeriesKMeans
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

import numpy as np

series = [np.array(serie).reshape(-1, 1) for serie in series_por_vivienda]

# Agrupar por vivienda y convertir en series
series_por_vivienda = (
    df.sort_values(by=['cups', 'fecha', 'hora'])
      .groupby('cups')['consumo_kWh']
      .apply(list)
)
# Semilla para reproducibilidad
seed = 0
np.random.seed(seed)

# Cargar datos: aquí usamos el dataset "Trace" de tslearn
X_train, y_train, X_test, y_test = CachedDatasets().load_dataset("Trace")

# (Opcional) Resample: para trabajar con series de igual longitud (por ejemplo, 50 puntos)
X_train = TimeSeriesResampler(sz=50).fit_transform(X_train)

# Crear el pipeline
pipeline = Pipeline([
    ('scaler', TimeSeriesScalerMeanVariance()),  # Normaliza las series (media 0 y varianza 1)
    ('clustering', TimeSeriesKMeans(n_clusters=3, metric="dtw", random_state=seed))
])

# Ajustar el pipeline a los datos
pipeline.fit(X_train)

# Predecir las etiquetas de cluster para cada serie
labels = pipeline.predict(X_train)
print("Etiquetas de cluster:", labels)

# Ejemplo de visualización: graficar los centroides
centroids = pipeline.named_steps['clustering'].cluster_centers_
plt.figure(figsize=(10, 6))
for idx, centroid in enumerate(centroids):
    plt.plot(centroid.ravel(), label=f'Cluster {idx}')
plt.title("Centroides de los clusters (TimeSeriesKMeans con DTW)")
plt.xlabel("Tiempo")
plt.ylabel("Valor escalado")
plt.legend()
plt.show()
